In [173]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [174]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [175]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Benchmark

## Import dependencies

In [176]:
import os
import pandas as pd

from datetime import datetime, timezone
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from docs.examples.benchmark_utils import clear_directory, create_averaged_dfs_dict, populate_benchmark_report
from virny.datasets.data_loaders import CompasWithoutSensitiveAttrsDataset, ACSEmploymentDataset
from virny.user_interfaces.metrics_computation_interfaces import compute_metrics_multiple_runs
from virny.utils.custom_initializers import create_config_obj, read_model_metric_dfs

## COMPAS_Without_Sensitive_Attributes Dataset

## Initialize Input Variables

### Create a models config

In [177]:
models_config = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=20,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
    # 'LogisticRegression': LogisticRegression(C=1,
    #                                          max_iter=50,
    #                                          penalty='l2',
    #                                          solver='newton-cg'),
    #     'RandomForestClassifier': RandomForestClassifier(max_depth=4,
    #                                                      max_features=0.6,
    #                                                      min_samples_leaf=1,
    #                                                      n_estimators=50),
    #     'XGBClassifier': XGBClassifier(learning_rate=0.1,
    #                                    max_depth=5,
    #                                    n_estimators=20),
}

### Create a Dataset class

In [178]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path=os.path.join('virny', 'datasets', 'COMPAS.csv'))
dataset.X_data[dataset.X_data.columns[:6]].head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45
0,0.0,-2.340451,1.0,-15.010999,1,0
1,0.0,0.000000,0.0,0.000000,1,0
2,0.0,0.000000,0.0,0.000000,0,0
3,0.0,0.000000,0.0,6.000000,1,0
4,0.0,0.000000,0.0,7.513697,1,0


### Create a config object

In [179]:
ROOT_DIR = os.path.join('docs', 'examples')
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_compas_config.yaml')
config_yaml_content = """
dataset_name: COMPAS_Without_Sensitive_Attributes
test_set_fraction: 0.2
bootstrap_fraction: 0.8
n_estimators: 10
#runs_seed_lst: [100, 200, 300, 400, 500, 600]
runs_seed_lst: [100, 200]
sensitive_attributes_dct: {'sex': 0, 'race': 'Caucasian', 'sex&race': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [180]:
config = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH1 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_results', config.dataset_name)
STANDARD_RESULTS_DIR_PATH1 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'standard_results', config.dataset_name)

In [181]:
clear_directory(SAVE_RESULTS_DIR_PATH1)

Directory is cleared


## Subgroup Metrics Computation

In [182]:
multiple_run_metrics_dct = compute_metrics_multiple_runs(dataset, config, models_config, SAVE_RESULTS_DIR_PATH1, debug_mode=False)

Multiple runs progress:   0%|          | 0/2 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-02-04 13:35:47 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

...

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Model random_state:  201
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)




2023-02-04 13:35:48 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

2023-02-04 13:35:48 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 13:35:49 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


In [183]:
sample_model_metrics_df = multiple_run_metrics_dct[list(models_config.keys())[0]]
sample_model_metrics_df.head(20)

,Metric,overall,sex_priv,sex_dis,race_priv,race_dis,sex&race_priv,sex&race_dis,Model_Seed,Model_Name,Run_Number
0,Mean,0.530392,0.563868,0.522618,0.595395,0.486264,0.594340,0.474811,101,DecisionTreeClassifier,Run_1
1,Std,0.070279,0.076096,0.068929,0.068199,0.071692,0.085854,0.072405,101,DecisionTreeClassifier,Run_1
2,IQR,0.075941,0.081982,0.074538,0.072724,0.078125,0.091328,0.078887,101,DecisionTreeClassifier,Run_1
3,Entropy,0.000000,0.000000,0.000000,0.000000,0.246619,0.000000,0.241298,101,DecisionTreeClassifier,Run_1
4,Jitter,0.167382,0.197320,0.160430,0.147801,0.180675,0.191919,0.176190,101,DecisionTreeClassifier,Run_1
5,Per_Sample_Accuracy,0.655303,0.652764,0.655893,0.655269,0.655326,0.634091,0.652703,101,DecisionTreeClassifier,Run_1
6,Label_Stability,0.776515,0.731658,0.786931,0.800468,0.760254,0.745455,0.768726,101,DecisionTreeClassifier,Run_1
7,TPR,0.577002,0.471429,0.594724,0.408805,0.658537,0.280000,0.671378,101,DecisionTreeClassifier,Run_1
8,TNR,0.752197,0.782946,0.743182,0.817164,0.694352,0.761905,0.663830,101,DecisionTreeClassifier,Run_1
9,PPV,0.665877,0.540984,0.686981,0.570175,0.701299,0.318182,0.706320,101,DecisionTreeClassifier,Run_1


## Create a Benchmark Report

In [184]:
models_metrics_dct = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH1, model_names=models_config.keys())
standard_models_metrics_dct = read_model_metric_dfs(STANDARD_RESULTS_DIR_PATH1, model_names=models_config.keys())

avg_models_metrics_dct = create_averaged_dfs_dict(models_metrics_dct)
avg_standard_models_metrics_dct = create_averaged_dfs_dict(standard_models_metrics_dct)

In [185]:
report_df = pd.DataFrame()
report_df = populate_benchmark_report(report_df, avg_models_metrics_dct, avg_standard_models_metrics_dct,
                                      config.dataset_name, config.sensitive_attributes_dct)

In [186]:
report_df

,Dataset,Model,Subgroup,Delta%_Accuracy,Delta%_Entropy,Delta%_F1,Delta%_FNR,Delta%_FPR,Delta%_IQR,Delta%_Jitter,Delta%_Label_Stability,Delta%_Mean,Delta%_PPV,Delta%_Per_Sample_Accuracy,Delta%_Positive-Rate,Delta%_Selection-Rate,Delta%_Std,Delta%_TNR,Delta%_TPR
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,overall,0.663,0.000,1.278,-1.257,-0.210,0.706,-4.037,5.011,0.145,1.091,0.805,0.187,0.663,0.226,0.210,1.257
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_priv,0.923,10.467,0.829,-2.273,0.132,0.670,-4.271,5.143,0.543,-0.870,2.482,5.909,0.352,0.437,-0.132,2.273
2,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_dis,0.602,0.000,1.337,-1.024,-0.279,0.715,-3.982,4.979,0.055,1.487,0.415,-0.742,0.731,0.177,0.279,1.024
3,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_priv,0.227,0.000,1.787,-2.002,0.503,1.171,-3.682,4.525,0.049,1.142,0.289,1.797,1.224,0.307,-0.503,2.002
4,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_dis,0.971,7.759,1.349,-1.220,-0.555,0.388,-4.258,5.330,0.059,1.297,1.160,-0.305,0.628,0.173,0.555,1.220
5,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_priv,-1.535,21.545,-0.970,-5.000,5.988,1.284,-3.732,4.374,-0.674,-7.393,-1.254,27.375,4.376,0.595,-5.988,5.000
6,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_dis,0.573,7.741,1.127,-1.413,0.637,0.431,-4.166,5.240,-0.247,0.733,0.245,0.883,1.356,0.144,-0.637,1.413


## Folktables [GA, 2018] Dataset

In [187]:
dataset2 = ACSEmploymentDataset(state=['GA'], year=2018, root_dir=os.path.join('virny', 'datasets'), with_nulls=False, subsample=20_000)
dataset2.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
78002,3,2,0,1,2,1,2,2,2,2,2,0,1,1,22,60
62790,1,2,0,1,2,1,2,2,2,1,1,0,3,2,19,55
91863,5,0,1,1,2,1,2,2,2,2,1,2,1,4,8,11
70172,1,4,0,1,2,1,2,2,2,2,9,0,1,2,20,24
56391,3,2,0,1,2,1,2,2,2,1,1,0,1,2,19,33


In [188]:
models_config2 = {
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion='gini',
                                                     max_depth=20,
                                                     max_features=0.6,
                                                     min_samples_split=0.1),
    # 'LogisticRegression': LogisticRegression(C=1,
    #                                          max_iter=50,
    #                                          penalty='l2',
    #                                          solver='newton-cg'),
    #     'RandomForestClassifier': RandomForestClassifier(max_depth=4,
    #                                                      max_features=0.6,
    #                                                      min_samples_leaf=1,
    #                                                      n_estimators=50),
    #     'XGBClassifier': XGBClassifier(learning_rate=0.1,
    #                                    max_depth=5,
    #                                    n_estimators=20),
}

In [189]:
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_compas_config.yaml')
config_yaml_content = """
dataset_name: Folktables_GA_2018
test_set_fraction: 0.2
bootstrap_fraction: 0.8
n_estimators: 100
runs_seed_lst: [100, 200]
#runs_seed_lst: [100, 200, 300, 400, 500, 600]
sensitive_attributes_dct: {'SEX': '1', 'RAC1P': '1', 'SEX&RAC1P': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

config2 = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH2 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_results', config2.dataset_name)
STANDARD_RESULTS_DIR_PATH2 = os.path.join(ROOT_DIR, 'results', 'benchmark', 'standard_results', config2.dataset_name)

In [190]:
clear_directory(SAVE_RESULTS_DIR_PATH2)

Directory is cleared


In [191]:
multiple_run_metrics_dct2 = compute_metrics_multiple_runs(dataset2, config2, models_config2, SAVE_RESULTS_DIR_PATH2, debug_mode=False)

Multiple runs progress:   0%|          | 0/2 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Model random_state:  101
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-02-04 13:35:53 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

...

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

##############################  [Model 1 / 1] Analyze DecisionTreeClassifier  ##############################
Model random_state:  201
Baseline X_train shape:  (16000, 16)
Baseline X_test shape:  (4000, 16)




2023-02-04 13:36:33 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/100 [00:00<?, ?it/s]

2023-02-04 13:36:36 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
2023-02-04 13:36:45 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics


In [192]:
sample_model_metrics_df = multiple_run_metrics_dct2[list(models_config2.keys())[0]]
sample_model_metrics_df.head(20)

,Metric,overall,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis,SEX&RAC1P_priv,SEX&RAC1P_dis,Model_Seed,Model_Name,Run_Number
0,Mean,0.554042,0.534603,0.573327,0.546373,0.569560,0.514931,0.565602,101,DecisionTreeClassifier,Run_1
1,Std,0.058806,0.058721,0.058890,0.058148,0.060138,0.058293,0.060726,101,DecisionTreeClassifier,Run_1
2,IQR,0.070691,0.069008,0.072361,0.071291,0.069479,0.070140,0.072253,101,DecisionTreeClassifier,Run_1
3,Entropy,0.000000,0.000000,0.000000,0.000000,0.083091,0.000000,0.000000,101,DecisionTreeClassifier,Run_1
4,Jitter,0.045426,0.037950,0.052843,0.043167,0.049997,0.037194,0.060828,101,DecisionTreeClassifier,Run_1
5,Per_Sample_Accuracy,0.821840,0.844257,0.799602,0.813414,0.838889,0.835356,0.815360,101,DecisionTreeClassifier,Run_1
6,Label_Stability,0.939890,0.948474,0.931375,0.942092,0.935435,0.949380,0.923890,101,DecisionTreeClassifier,Run_1
7,TPR,0.845938,0.842672,0.849475,0.823387,0.897248,0.822289,0.900356,101,DecisionTreeClassifier,Run_1
8,TNR,0.815801,0.859962,0.774978,0.816284,0.814910,0.860182,0.766129,101,DecisionTreeClassifier,Run_1
9,PPV,0.787278,0.839957,0.737589,0.794553,0.772512,0.855799,0.744118,101,DecisionTreeClassifier,Run_1


In [193]:
models_metrics_dct2 = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH2, model_names=models_config2.keys())
standard_models_metrics_dct2 = read_model_metric_dfs(STANDARD_RESULTS_DIR_PATH2, model_names=models_config2.keys())

avg_models_metrics_dct2 = create_averaged_dfs_dict(models_metrics_dct2)
avg_standard_models_metrics_dct2 = create_averaged_dfs_dict(standard_models_metrics_dct2)

## Create a Benchmark Report

In [194]:
report_df = populate_benchmark_report(report_df, avg_models_metrics_dct2, avg_standard_models_metrics_dct2,
                                      config2.dataset_name, config2.sensitive_attributes_dct)

In [197]:
filename = f'benchmark_report_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}'
report_df.to_csv(os.path.join(ROOT_DIR, 'results', 'benchmark', 'benchmark_reports', f'{filename}.csv'), index=False)

In [196]:
report_df

,Dataset,Model,Subgroup,Delta%_Accuracy,Delta%_Entropy,Delta%_F1,Delta%_FNR,Delta%_FPR,Delta%_IQR,Delta%_Jitter,Delta%_Label_Stability,Delta%_Mean,Delta%_PPV,Delta%_Per_Sample_Accuracy,Delta%_Positive-Rate,Delta%_Selection-Rate,Delta%_Std,Delta%_TNR,Delta%_TPR
0,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,overall,0.663,0.000,1.278,-1.257,-0.210,0.706,-4.037,5.011,0.145,1.091,0.805,0.187,0.663,0.226,0.210,1.257
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_priv,0.923,10.467,0.829,-2.273,0.132,0.670,-4.271,5.143,0.543,-0.870,2.482,5.909,0.352,0.437,-0.132,2.273
2,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex_dis,0.602,0.000,1.337,-1.024,-0.279,0.715,-3.982,4.979,0.055,1.487,0.415,-0.742,0.731,0.177,0.279,1.024
3,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_priv,0.227,0.000,1.787,-2.002,0.503,1.171,-3.682,4.525,0.049,1.142,0.289,1.797,1.224,0.307,-0.503,2.002
4,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,race_dis,0.971,7.759,1.349,-1.220,-0.555,0.388,-4.258,5.330,0.059,1.297,1.160,-0.305,0.628,0.173,0.555,1.220
5,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_priv,-1.535,21.545,-0.970,-5.000,5.988,1.284,-3.732,4.374,-0.674,-7.393,-1.254,27.375,4.376,0.595,-5.988,5.000
6,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,sex&race_dis,0.573,7.741,1.127,-1.413,0.637,0.431,-4.166,5.240,-0.247,0.733,0.245,0.883,1.356,0.144,-0.637,1.413
7,Folktables_GA_2018,DecisionTreeClassifier,overall,-0.575,0.000,-0.770,0.110,0.875,-1.003,0.830,-1.082,0.080,-1.291,-0.610,1.701,-0.012,-0.266,-0.875,-0.110
8,Folktables_GA_2018,DecisionTreeClassifier,SEX_priv,0.075,0.000,0.311,-0.947,0.700,-0.870,0.719,-0.852,-0.664,-0.325,-0.083,1.499,1.053,-0.128,-0.700,0.947
9,Folktables_GA_2018,DecisionTreeClassifier,SEX_dis,-1.063,0.000,-1.837,1.267,0.695,-1.135,0.889,-1.247,0.690,-2.172,-0.988,1.871,-1.063,-0.397,-0.695,-1.267
